<a href="https://colab.research.google.com/github/regina-fonseca-avila/data_science/blob/main/table_visualization_pandas/table_visualization_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# **VISUALIZAÇÃO DE TABELAS PANDAS**

As vezes precisamos de fazer uma apresentação rápida e aqui mostramos como deixar suas tabelas bonitas e formas de destacar informações importantes em uma análise mesmo quando está utilizando um notebook.

Vamos nos basear em duas referências

[1] https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

[2] https://towardsdatascience.com/style-pandas-dataframe-like-a-master-6b02bf6468b0

para ver como utilizar o Styler Class e trazer alguns exemplos.

- Styler cria uma tabela HTML e aproveita a linguagem CSS pra manipular parâmetros como

  - cor, font, borda, fundo

Styling deve ser utilizado depois do dados no DataFrame serem processados

É ideal para ser utilizadas nas tabelas (dataframes) pequenos que já trazem dados consolidados.

As duas referências trazem exemplos muito fáceis de serem compreendidos. Para ver como se **formata valores** veja a referência [2].

Vamos trazer um exemplo utilizando um dataset público, faremos um groupby para mostrar os dados de vendas consolidado por hora e por dia da semana e vamos mostrar como utilizando o style podemos destacar informações imprtantes dos dados de forma muito simples.

# Dados

**UCI machine learning repository**

Online Retail Data Set

Mais informações [aqui](https://archive.ics.uci.edu/ml/datasets/online+retail#).

In [2]:
# read data and first lines
file_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
            
online = pd.read_excel(file_path)

online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
online.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [4]:
# cleaning dataset
online = online[online.UnitPrice > 0]
online = online[online.Quantity > 0]
online = online[~online.CustomerID.isna()]

## Criando colunas de dia da semana e hora da compra

In [5]:
# The day of the week with Monday=0, Sunday=6
online['DayOfWeek'] = online['InvoiceDate'].dt.weekday

# The hours of the datetime
online['Hour'] = online['InvoiceDate'].dt.hour

online.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,DayOfWeek,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2,8


# Número de compra únicas por hora e dia da semana

In [6]:
n_tickets_by_week_and_hour = online.groupby(['DayOfWeek','Hour'], as_index=False).agg({'InvoiceNo':'nunique'})
n_tickets_by_week_and_hour.head()

,DayOfWeek,Hour,InvoiceNo
0,0,7,4
1,0,8,85
2,0,9,251
3,0,10,322
4,0,11,340


In [7]:
unique_tickets_by_week_and_hour = pd.pivot_table(n_tickets_by_week_and_hour, 
                                          index = 'DayOfWeek',
                                          columns = 'Hour', 
                                          values = 'InvoiceNo',
                                          fill_value=0)

unique_tickets_by_week_and_hour

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


# Estilo da Tabela

In [8]:
s = unique_tickets_by_week_and_hour.style

cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


- O fundo amarelo quando movimentamos o mouse pode ser útil durante uma apresentação para facilitar que outras pessoas vejam a informação que você quer mostrar.

# Título

In [9]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


# Destacando valores iguais a zero

In [10]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)

# style zero
def style_zero(v, props=''):
    return props if v == 0 else None
s.applymap(style_zero, props='color:red;')

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


- Aqui podemos ver dia e horários em que não ocorreram vendas
- No lugar de mostrar os zeros em vermelho também poderiamos mostrar em vermelho valores menor que um mímino estabelecido.

# Destacando os valores maiores que 500 e deixando os demais mais claros

In [11]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)

# style zero
def style_zero(v, props=''):
    return props if v == 0 else None
s.applymap(style_zero, props='color:red;')

# unique_tickets_by_week_and_hour
s.applymap(lambda v: 'opacity: 20%;' if (v < 500) else None)


Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


# Valor máximo em cada linha

In [12]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)


s.highlight_max(color='lightgreen', axis = 1)

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


- Interessante notar que o valor máximo sempre aconteceu as 12 horas.

# Valor mínimo em cada coluna

In [13]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)


s.highlight_min(color='#FF7F7F', axis = 0)

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


- Se tivermos o valor mímino se repetindo vai destacar o fundo em todos os pontos.
- Valores mínimos se concentram no dia 6, Domingo e as 6 h

# Maior Valor Geral

In [14]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)

# highlight max

s.highlight_max(color='lightgreen', axis = None)

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
DayOfWeek,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0


- Quarta (dia 2) as 12 horas é o dia com o maior número de vendas.

# Tabela com margens - total da linha e total da coluna

In [15]:
unique_tickets_by_week_and_hour = pd.pivot_table(n_tickets_by_week_and_hour, 
                                          index = 'DayOfWeek',
                                          columns = 'Hour', 
                                          values = 'InvoiceNo',
                                          fill_value=0,
                                          margins=True, 
                                          margins_name='All',
                                          aggfunc='sum'
                                          )

unique_tickets_by_week_and_hour


Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,All
DayOfWeek,,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0,2863
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0,3184
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1,3455
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16,4033
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1,2829
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0,2169
All,1,29,555,1393,2226,2277,3130,2636,2274,2037,1100,544,169,144,18,18533


In [16]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)


# backgroud column All
slice_ = ['All']
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)

# backgroud row All
idx = pd.IndexSlice
slice_ = idx['All',:]
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)


# vertical line
s.set_table_styles({
    ('All'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)


# horizontal line
s.set_table_styles({
    ('All'): [{'selector': '', 'props': 'border-top: 1px solid white'},
                               {'selector': 'td', 'props': 'border-top: 1px solid #000066'}]
}, overwrite=False, axis=1)

Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,All
DayOfWeek,,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0,2863
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0,3184
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1,3455
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16,4033
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1,2829
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0,2169
All,1,29,555,1393,2226,2277,3130,2636,2274,2037,1100,544,169,144,18,18533


- Muito melhor ver os resultados agora.

# Máximo entre as soma das linhas (dias da semana)

In [17]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)


# backgroud column All
slice_ = ['All']
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)

# backgroud row All
idx = pd.IndexSlice
slice_ = idx['All',:]
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)


# vertical line
s.set_table_styles({
    ('All'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)


# horizontal line
s.set_table_styles({
    ('All'): [{'selector': '', 'props': 'border-top: 1px solid white'},
                               {'selector': 'td', 'props': 'border-top: 1px solid #000066'}]
}, overwrite=False, axis=1)

# highlight max in row All

idx = pd.IndexSlice
slice_ = idx[idx[0:6], idx['All']]

s.highlight_max(color='lightgreen', axis = 0, subset=slice_)



Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,All
DayOfWeek,,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0,2863
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0,3184
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1,3455
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16,4033
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1,2829
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0,2169
All,1,29,555,1393,2226,2277,3130,2636,2274,2037,1100,544,169,144,18,18533


- Dia 3 (quinta) é o dia com mais vendas.

# Mínimo entre as soma das colunas (dias da semana)

In [18]:
s = unique_tickets_by_week_and_hour.style

# table style
cell_hover = { # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000076; color: white;'
}

s.set_table_styles([cell_hover, index_names, headers])

# title
s.set_caption("Count Invoices")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: upper; font-size:2.25em; font-weight:bold'
 }],overwrite=False)


# backgroud column All
slice_ = ['All']
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)

# backgroud row All
idx = pd.IndexSlice
slice_ = idx['All',:]
s.set_properties(**{'background-color': 'lightblue'}, subset=slice_)


# vertical line
s.set_table_styles({
    ('All'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)


# horizontal line
s.set_table_styles({
    ('All'): [{'selector': '', 'props': 'border-top: 1px solid white'},
                               {'selector': 'td', 'props': 'border-top: 1px solid #000066'}]
}, overwrite=False, axis=1)

# highlight max in row All

idx = pd.IndexSlice
slice_ = idx[idx['All'], idx[6:20]] # [6:20] = name of first column: name of last column

s.highlight_min(color='#FF7F7F', axis = 1, subset=slice_)


Hour,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,All
DayOfWeek,,,,,,,,,,,,,,,,
0,0,4,85,251,322,340,480,436,374,317,184,70,0,0,0,2863
1,0,5,115,290,393,405,531,475,371,345,163,88,3,0,0,3184
2,0,5,116,290,434,413,611,493,417,371,223,79,0,2,1,3455
3,1,9,119,294,442,370,586,479,445,421,309,245,164,133,16,4033
4,0,6,120,266,410,364,463,397,361,253,115,62,2,9,1,2829
6,0,0,0,2,225,385,459,356,306,330,106,0,0,0,0,2169
All,1,29,555,1393,2226,2277,3130,2636,2274,2037,1100,544,169,144,18,18533


- O pior horário é 6 horas.

# Observações

- Não há dados para o dia 5, sábado. É algo que podia ser investigado.

- Teria ficado mais fácil analisar se tivesse colocado o nome dos dias da semana

- Poderia ampliar a análise considerando os valores das compras, o número de clientes


# Conclusão

Utilitilizando Pandas, groupby, pivot_table e style podemos extrair informações importantes dos dados de forma clara e muito fácil de visualizar.